In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
import numpy as np
import torch

# Test MQTT

## IP Address, Port Number, and Topics

In [ ]:
from compas_eve import Publisher, Message, Topic, Subscriber
from compas_eve.mqtt import MqttTransport
tx = MqttTransport(host = "localhost", port=1883)

## Publish a text message

In [ ]:
publisher = Publisher(Topic("/robot/hello"), transport=tx)
publisher.publish({"msg": "Hello World"})
time.sleep(1)

## Publish numpy array

In [ ]:
publisher = Publisher(Topic("/robot/hello"), transport=tx)
publisher.publish({"msg": np.array([0, 1, 2])})
time.sleep(1)

## Publish a mesh shelf

In [ ]:
import trimesh
from robofab.mqtt import encoding_mesh
mesh = trimesh.load("/RoboFab/data/world/shelf.obj")
tx = MqttTransport(host = "localhost", port=1883)
publisher = Publisher(Topic("/robot/hello"), transport=tx)
publisher.publish({"msg": encoding_mesh(mesh)})
time.sleep(1)

# 1. Load your first robot

## 1.1 Curobo common import

In [ ]:
# cuRobo
from curobo.cuda_robot_model.cuda_robot_model import CudaRobotModel, CudaRobotModelConfig
from curobo.types.base import TensorDeviceType
from curobo.types.robot import RobotConfig
from curobo.util_file import get_robot_path, join_path, load_yaml

# convenience function to store tensor type and device
tensor_args = TensorDeviceType()

## 1.2 Load from the current file

In [ ]:
# the path of the yml file must be absolute
from robofab import ROBOFAB_DATA_DIR
config_file = load_yaml(ROBOFAB_DATA_DIR+ "/robot/franka/fr3_franka.yml")["robot_cfg"]
config_kinematics = config_file['kinematics']
config_kinematics["urdf_path"] = ROBOFAB_DATA_DIR + "/robot/franka/" + config_kinematics["urdf_path"]
config_kinematics["collision_spheres"] = ROBOFAB_DATA_DIR + "/robot/franka/" + config_kinematics["collision_spheres"]
robot_cfg = RobotConfig.from_dict(config_file, tensor_args)
kin_model = CudaRobotModel(robot_cfg.kinematics)

## 1.3 Visualize robot

In [ ]:
# Run file robot_viewer.py
from robofab.mqtt import publish_robot_trajs, publish_spheres, publish_clear
retract_q = kin_model.retract_config
q_finger = np.hstack([retract_q.cpu().numpy().reshape(-1), [0.04, 0.04]]).reshape(1, -1)
publish_clear()
publish_robot_trajs(q_finger)
publish_spheres(kin_model, retract_q) # Visualize robot as spheres

## 1.4 load from curobo folder  

In [ ]:
# this example loads urdf from a configuration file, you can also load from path directly
# load a urdf, the base frame and the end-effector frame:
config_file2 = load_yaml(join_path(get_robot_path(), "franka.yml"))["robot_cfg"]
robot_cfg2 = RobotConfig.from_dict(config_file2, tensor_args)
kin_model2 = CudaRobotModel(robot_cfg.kinematics)

### All supported robots

In [ ]:
from os import listdir
from os.path import isfile, join
files = [f for f in listdir(get_robot_path()) if isfile(join(get_robot_path(), f))]
print(files)

### Load UR 10e

In [ ]:
# UR 10e
config_file2 = load_yaml(join_path(get_robot_path(), "ur10e.yml"))["robot_cfg"]
robot_cfg2 = RobotConfig.from_dict(config_file2, tensor_args)
kin_model2 = CudaRobotModel(robot_cfg2.kinematics)
print(kin_model2.get_dof())

### Load dual UR 10e

In [ ]:
# dual UR 10e
config_file2 = load_yaml(join_path(get_robot_path(), "dual_ur10e.yml"))["robot_cfg"]
robot_cfg2 = RobotConfig.from_dict(config_file2, tensor_args)
kin_model2 = CudaRobotModel(robot_cfg2.kinematics)
print(kin_model2.get_dof())

In [ ]:
from robofab.mqtt import publish_clear
publish_clear()
publish_spheres(kin_model2, kin_model2.retract_config)

# 2. Forward Kinematics

## 2.1 Single forward kinematics

In [ ]:
from scipy.spatial.transform import Rotation as R 
from robofab.mqtt import publish_robot_trajs, publish_frames
import numpy as np

# random joint angle
q = torch.rand((1, kin_model.get_dof()), **(tensor_args.as_torch_dict()))
# forward kinematics
ee_pose = kin_model.get_state(q)

# to frame
pos = ee_pose.ee_position.cpu().numpy()[0]
quat = ee_pose.ee_quaternion.cpu().numpy()[0]
rot = R.from_quat(quat, scalar_first = True).as_matrix()
frame = np.eye(4)
frame[0:3, 0:3] = rot
frame[0:3, 3] = pos
frames = frame.reshape(1, 4, 4)

# publish to MQTT
q_finger = np.hstack([q.cpu().numpy().reshape(-1), [0.04, 0.04]]).reshape(1, -1)
publish_clear()
publish_robot_trajs(q_finger)
publish_frames(frames)

## 2.2 Batch Forward Kinematics

In [ ]:
from scipy.spatial.transform import Rotation as R 
import numpy as np

qs = torch.rand((1000, kin_model.get_dof()), **(tensor_args.as_torch_dict()))
ee_poses = kin_model.get_state(qs)
pos = ee_poses.ee_position.cpu().numpy()
quat = ee_poses.ee_quaternion.cpu().numpy()
rot = R.from_quat(quat, scalar_first = True).as_matrix()
frames = np.zeros((qs.shape[0], 4, 4))
frames[:, 3, 3] = 1
frames[:, 0:3, 0:3] = rot
frames[:, 0:3, 3] = pos

# publish to MQTT
publish_robot_trajs(qs.cpu().numpy())
publish_frames(frames)

## 3. Collision detection

## 3.1 Create collision world

### From code

In [ ]:
from curobo.geom.types import WorldConfig, Cuboid, Mesh, Capsule, Cylinder, Sphere
from curobo.util_file import get_assets_path, join_path, get_configs_path

obstacle_1 = Cuboid(
     name="cube_1",
     pose=[0.0, 0.0, 0.0, 0.043, -0.471, 0.284, 0.834],
     dims=[0.2, 1.0, 0.2],
     color=[0.8, 0.0, 0.0, 1.0],
 )

obstacle_2 = Capsule(
   name="capsule",
   radius=0.2,
   base=[0, 0, 0],
   tip=[0, 0, 0.5],
   pose=[0.0, 1, 0.0, 0.043, -0.471, 0.284, 0.834],
   color=[0, 1.0, 0, 1.0],
)

obstacle_3 = Cylinder(
   name="cylinder_1",
   radius=0.2,
   height=0.1,
   pose=[0.0, 2, 0.0, 0.043, -0.471, 0.284, 0.834],
   color=[0, 1.0, 0, 1.0],
)

obstacle_4 = Sphere(
   name="sphere_1",
   radius=0.2,
   pose=[0.0, 3, 0.0, 0.043, -0.471, 0.284, 0.834],
   color=[0, 1.0, 0, 1.0],
)

world_model = WorldConfig(
   cuboid=[obstacle_1],
   capsule=[obstacle_2],
   cylinder=[obstacle_3],
   sphere=[obstacle_4],
)

# send mqtt
from robofab.mqtt import publish_world, publish_clear
publish_clear()
publish_world(world_model)

### 3.1.2 From dictionary

In [ ]:
# cuRobo
from curobo.geom.types import WorldConfig
# create a world from a dictionary of objects
# cuboid: {} # dictionary of objects that are cuboids
# mesh: {} # dictionary of objects that are meshes
world_config = {
    "cuboid": {
        "table": {"dims": [2, 2, 0.2], "pose": [0.4, 0.0, -0.1, 1, 0, 0, 0]},
        "cube_1": {"dims": [0.1, 0.1, 0.2], "pose": [0.4, 0.0, 0.5, 1, 0, 0, 0]},
    }
}

world_model = WorldConfig.from_dict(world_config)

# publish to MQTT
publish_clear()
publish_world(world_model)

## 3.1.3 From file

In [ ]:
from curobo.geom.types import WorldConfig
print(get_assets_path())
world_config = load_yaml(join_path(ROBOFAB_DATA_DIR, "world/collision_shelf.yml"))
world_model = WorldConfig.from_dict(world_config)

# publish to MQTT
publish_clear()
publish_robot_trajs(kin_model.retract_config.cpu().numpy().reshape(1, -1))
publish_world(world_model)

## 3.3 Check self-collision and world collision

In [ ]:
from curobo.wrap.model.robot_world import RobotWorld, RobotWorldConfig
robot_world_config = RobotWorldConfig.load_from_config(robot_cfg, world_config, collision_activation_distance=0.0)
curobo_fn = RobotWorld(robot_world_config)
q_s = curobo_fn.sample(1000, mask_valid=False)
d_world, d_self = curobo_fn.get_world_self_collision_distance_from_joints(q_s)
flag = torch.logical_and(d_world < 1E-6, d_self < 1E-6)
q_s = q_s[flag, :]

# publish to MQTT
publish_clear()
publish_world(world_model)
publish_robot_trajs(q_s.cpu().numpy())


## 3. Inverse Kinematics

## 3.1 IK without collision checking

In [ ]:
# cuRobo
from curobo.types.math import Pose
from curobo.types.robot import RobotConfig
from curobo.util_file import get_robot_configs_path, join_path, load_yaml
from curobo.wrap.reacher.ik_solver import IKSolver, IKSolverConfig

ik_config = IKSolverConfig.load_from_robot_config(
    robot_cfg,
    None,
    rotation_threshold=0.05,
    position_threshold=0.005,
    num_seeds=20,
    tensor_args=tensor_args,
    use_cuda_graph=True,
)
ik_solver = IKSolver(ik_config)

### Pick the cube on the ground

In [ ]:
from scipy.spatial.transform import Rotation as R 
import numpy as np
rot = R.from_matrix([[1, 0, 0], 
                     [0, -1, 0], 
                     [0, 0, -1]])
quat = rot.as_quat(scalar_first = True)
xyz = [0, -0.52, 0.02]
goal_pose_cube_ground = Pose.from_list([*xyz, *quat], tensor_args)
q = ik_solver.solve_single(goal_pose_cube_ground).js_solution.position.reshape(-1).cpu().numpy()[:7]

# publish to MQTT
publish_clear()
publish_world(world_model)
publish_robot_trajs(q.reshape(1, -1), finger_width=0.02)

## 3.2 Check IK with collision avoidance

In [ ]:
ik_config = IKSolverConfig.load_from_robot_config(
    robot_cfg,
    world_model,
    rotation_threshold=0.05,
    position_threshold=0.005,
    num_seeds=20,
    self_collision_check=True,
    self_collision_opt=True,
    tensor_args=tensor_args,
    use_cuda_graph=True,
)
ik_solver_collision = IKSolver(ik_config)

### Pick the cube on the ground with collision avoidance

In [ ]:
q = ik_solver_collision.solve_single(goal_pose_cube_ground.clone()).js_solution.position.reshape(-1).cpu().numpy()[:7]

# publish to MQTT
publish_robot_trajs(q.reshape(1, -1), finger_width=0.02)

### Exercise: pick the cube on the shelf with collision

In [ ]:
# insert your code here

# 4. Robot Motion Planning

## 4.1 start_js -> goal_pose

### Initialize Motion Planner

In [ ]:
from curobo.types.math import Pose
from curobo.types.robot import JointState
from curobo.wrap.reacher.motion_gen import MotionGen, MotionGenConfig, MotionGenPlanConfig

motion_gen_config = MotionGenConfig.load_from_robot_config(
    robot_cfg=robot_cfg,
    world_model=world_model,
    interpolation_dt=0.01,
)
motion_gen = MotionGen(motion_gen_config)
motion_gen.detach_object_from_robot()
motion_gen.warmup()


### Pick up the ground block

In [ ]:
start_state = JointState.from_position(kin_model.retract_config.reshape(1, -1), joint_names=kin_model.joint_names)
sol = motion_gen.plan_single(start_state=start_state, goal_pose=goal_pose_cube_ground.clone())
trajs = sol.get_interpolated_plan().position.cpu().numpy()

# publish to MQTT
publish_clear()
publish_world(world_model)
publish_robot_trajs(trajs=trajs, finger_width=0.02)

## 4.2 start_js -> goal_js

In [ ]:
start_state = JointState.from_position(kin_model.retract_config.reshape(1, -1), joint_names=kin_model.joint_names)
q_goal = ik_solver_collision.solve_single(goal_pose=goal_pose_cube_ground.clone()).js_solution[0].position[:, :7]
goal_state = JointState.from_position(q_goal, joint_names=kin_model.joint_names)
sol = motion_gen.plan_single_js(start_state=start_state, goal_state=goal_state)
trajs = sol.get_interpolated_plan().position.cpu().numpy()

# publish to MQTT
publish_robot_trajs(trajs=trajs, finger_width=0.02)

## Exercise: from pick up to place the cube on the shelf

In [ ]:
# insert your code here

# 5. Robot Grasp Objects

## 5.1 Compute grasp object transformation

In [ ]:
from trimesh.primitives import Box
from robofab.mqtt import publish_grasp_object

q = kin_model.retract_config.reshape(1, -1)
ee_pose = kin_model.get_state(q)

# to frame
pos = ee_pose.ee_position.cpu().numpy()[0]
quat = ee_pose.ee_quaternion.cpu().numpy()[0]
rot = R.from_quat(quat, scalar_first = True).as_matrix()
frame = np.eye(4)
frame[0:3, 0:3] = rot
frame[0:3, 3] = pos
frames = frame.reshape(1, 4, 4)

publish_clear()
publish_world(world_model)
publish_frames(frames)
publish_robot_trajs(trajs=q.cpu().numpy())

## 5.2 Attach external objects to the robot 

In [ ]:
from curobo.geom.types import Mesh
from curobo.geom.sphere_fit import SphereFitType

box = Box(extents=[0.04, 0.04, 0.1])
box = box.apply_translation([0, 0, 0.02])
publish_grasp_object(box)
box = box.apply_transform(frame)

grasp_object = Mesh(name = "box", vertices = box.vertices, faces = box.faces, pose = [0, 0, 0, 1, 0, 0 , 0])
start_state = JointState.from_position(q, joint_names = kin_model.joint_names)
kin_model.attach_external_objects_to_robot(start_state, [grasp_object], sphere_fit_type=SphereFitType.VOXEL_VOLUME_SAMPLE_SURFACE )
motion_gen.attach_external_objects_to_robot(start_state, [grasp_object], sphere_fit_type=SphereFitType.VOXEL_VOLUME_SAMPLE_SURFACE )
publish_robot_trajs(trajs=q.cpu().numpy())
publish_spheres(kin_model, q)

## 5.3 Exercise: place bar behind the red box

In [ ]:
# insert your code here